<a href="https://colab.research.google.com/github/micheleguidaa/robust-organ-scheduler/blob/main/robust_organ_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pianificazione Robusta dell'Orario di Prelievo in Interventi di Trapianto

**Studente:** Michele Guida  
**Corso:** Decision Support Systems  
**Repository:** [github.com/micheleguidaa/robust-organ-scheduler](https://github.com/micheleguidaa/robust-organ-scheduler)

## 1. Definizione del Problema

Il progetto affronta la scelta dell'orario di inizio ($t_{start}$) di un intervento di prelievo multi-organo da un singolo donatore.
L'obiettivo è minimizzare i ritardi di partenza dei mezzi di trasporto assegnati ai diversi organi, tenendo conto
della diversa importanza clinica e dell'incertezza sulla durata dell'intervento.

### Modello Matematico

Sia $\mathbf{T}$ l'insieme discreto degli orari di inizio possibili e $\mathbf{D} = \{d_{min}, d_{med}, d_{max}\}$ l'insieme delle possibili durate dell'intervento.

Definiamo il **tempo di attesa** $\delta_i$ per l'organo $i$ come il tempo che intercorre tra la disponibilità dell'organo (fine intervento) e la prima partenza utile del mezzo di trasporto $S_i^j$:

$$
\delta_{i}(t_{start},d) = \min_{j: S_{i}^{j} \ge t_{start}+d} (S_{i}^{j} - (t_{start}+d))
$$

La **funzione di costo** (ritardo totale pesato) per uno scenario è data da:

$$
F(t_{start},d) = \sum_{i \in \mathbf{I}} w_{i} \cdot \delta_{i}(t_{start},d)
$$

### Obiettivo: Ottimizzazione Robusta (Minimax)
Poiché la durata $d$ è incerta, adottiamo un approccio robusto che minimizza il costo nel *caso peggiore* (worst-case scenario):

$$
t_{start}^* = \arg \min_{t_{start} \in \mathbf{T}} \left( \max_{d \in \mathbf{D}} F(t_{start}, d) \right)
$$

## 2. Soluzione e Implementazione dell'Algoritmo

### Descrizione dell'Approccio
La funzione `robust_organ_scheduler` implementa una strategia di **enumerazione esplicita** per risolvere il problema di ottimizzazione robusta (Minimax). Dato che gli insiemi degli orari di inizio ($\mathbf{T}$), delle durate ($\mathbf{D}$) e delle partenze ($\mathbf{S}_i$) sono discreti e finiti, l'algoritmo esplora l'intero spazio delle soluzioni ammissibili.

Il flusso logico è il seguente:
1.  **Ciclo Esterno (Decisione):** Itera su tutti i possibili orari di inizio $t_{start} \in \mathbf{T}$.
2.  **Ciclo Interno (Incertezza):** Per ogni $t_{start}$, valuta tutti gli scenari di durata $d \in \mathbf{D}$ per identificare il *caso peggiore* (worst-case).
3.  **Calcolo del Costo:** Per ogni combinazione $(t_{start}, d)$, calcola il ritardo pesato sommando le attese $\delta_i$ di ciascun organo rispetto alla prima partenza utile disponibile.
4.  **Selezione Ottima:** Seleziona il $t_{start}$ che minimizza il costo del caso peggiore trovato.

### Analisi della Complessità Computazionale
La complessità dell'algoritmo dipende dalla cardinalità degli insiemi in input.
Definiamo:
* $|\mathbf{T}|$: numero di orari di inizio possibili.
* $|\mathbf{D}|$: numero di scenari di durata (nel nostro caso $K=3$).
* $ N $: numero di organi (nel nostro caso $N=5$).
* $|\mathbf{S}_i|$: numero di orari di partenza disponibili per l'organo $i$.

L'algoritmo esegue due cicli annidati principali. Al livello più interno, per ogni organo, viene eseguita una scansione lineare delle partenze disponibili per trovare la prima valida (`min(partenze_valide)`).

La complessità temporale asintotica è dunque:

$$
O\left( |\mathbf{T}| \cdot |\mathbf{D}| \cdot \sum_{i=1}^{N} |\mathbf{S}_i| \right)
$$

Poiché $|\mathbf{D}|$ e $N$ sono costanti molto piccole e fisse nel problema specifico, la complessità scala linearmente rispetto al numero di orari di inizio da valutare ($|\mathbf{T}|$) e al volume totale dei voli disponibili ($\sum |\mathbf{S}_i|$).

In [ ]:
import numpy as np
from datetime import datetime, date, timedelta
from typing import Sequence, Mapping, Tuple, Optional

def parse_orario(s: str) -> datetime:
    """
    Converte una stringa "HH:MM" in un oggetto datetime riferito alla data odierna
    """
    t = datetime.strptime(s, "%H:%M").time()
    return datetime.combine(date.today(), t)

def robust_organ_scheduler(
    orari_possibili_inizio_intervento: Sequence[datetime],
    durate_possibili_intervento: Sequence[timedelta],
    organi: Sequence[str],
    orari_partenza: Mapping[str, Sequence[datetime]],
    pesi: Mapping[str, float],
) -> Tuple[float, Optional[datetime], timedelta]:
    """
    Risolve il problema di scheduling robusto (minimax) restituendo:
    - valore della funzione obiettivo (float)
    - orario di inizio intervento ottimo (datetime) oppure None se non esiste soluzione
    - tempo di esecuzione dell'algoritmo (timedelta)
    """
    # Avvio del timer per misurare il tempo di esecuzione dell’algoritmo.
    inizio_esecuzione_algoritmo = datetime.now()

    # Inizializzazione del problema minimax: vogliamo minimizzare il massimo delle attese pesate
    funzione_obiettivo = np.inf
    migliore_orario_inizio_intervento = None

    # Valutazione di ogni possibile orario di inizio intervento
    for orario_inizio_intervento in orari_possibili_inizio_intervento:

        # Per questo orario consideriamo il peggior scenario tra tutte le durate
        somma_pesata_attese_max = -np.inf

        for durata_intervento in durate_possibili_intervento:
            # Calcolo della somma pesata delle attese per questa durata
            somma_pesata_attese = 0.0

            for organo in organi:
                # Orario di fine intervento per questa durata
                orario_fine_intervento = orario_inizio_intervento + durata_intervento

                # Selezioniamo solo le partenze non antecedenti alla fine dell'intervento
                partenze_valide = [
                    orario_partenza for orario_partenza in orari_partenza[organo]
                    if orario_partenza >= orario_fine_intervento
                ]

                # Se non ci sono partenze valide l’intervento diventa impossibile → costo infinito
                if not partenze_valide:
                    somma_pesata_attese = np.inf
                    break

                # Prima partenza disponibile
                prima_partenza_utile = min(partenze_valide)

                # Tempo di attesa rispetto alla fine dell’intervento
                attesa = prima_partenza_utile - orario_fine_intervento
                attesa_minuti = attesa.total_seconds() / 60

                # Aggiunta dell’attesa pesata secondo l’importanza dell’organo
                somma_pesata_attese += pesi[organo] * attesa_minuti

            # Aggiornamento dello scenario peggiore (max) per questo t_start
            if somma_pesata_attese > somma_pesata_attese_max:
                somma_pesata_attese_max = somma_pesata_attese

        # Aggiorniamo il best t_start globale confrontando i valori worst-case ottenuti
        if somma_pesata_attese_max < funzione_obiettivo:
            funzione_obiettivo = somma_pesata_attese_max
            migliore_orario_inizio_intervento = orario_inizio_intervento

    # Fine della misurazione del tempo totale di esecuzione.
    fine_esecuzione_algoritmo = datetime.now()

    # tempo_di_esecuzione quantifica il costo computazionale dell’enumerazione.
    tempo_di_esecuzione = fine_esecuzione_algoritmo - inizio_esecuzione_algoritmo

    return funzione_obiettivo, migliore_orario_inizio_intervento, tempo_di_esecuzione


In [ ]:
# Possibili orari di inizio dell’intervento (scelta robusta tra più slot)
orari_possibili_inizio_intervento_raw = ["10:30", "12:00"]
orari_possibili_inizio_intervento = [parse_orario(o) for o in orari_possibili_inizio_intervento_raw]

# Durate dell’intervento (scenario min, med, max) — usate per la valutazione worst-case
durata_min = 0
durata_max = 10
durata_med = (durata_min + durata_max) // 2

durate_possibili_intervento = [
    timedelta(minutes=durata_min),
    timedelta(minutes=durata_med),
    timedelta(minutes=durata_max),
]

# Lista degli organi da trasportare
organi = ["cuore", "polmoni", "rene_sx", "rene_dx", "fegato"]

# Per ciascun organo: possibili orari di partenza dei voli/mezzi disponibili
orari_partenza = {
    "cuore":   [parse_orario("10:45"), parse_orario("12:10")],
    "polmoni": [parse_orario("11:00"), parse_orario("12:10")],
    "rene_sx": [parse_orario("10:40"), parse_orario("12:10")],
    "rene_dx": [parse_orario("10:50"), parse_orario("12:15")],
    "fegato":  [parse_orario("11:10"), parse_orario("13:00")],
}

# Pesi che definiscono priorità/urgenza dell’organo nella funzione obiettivo
pesi = {"cuore": 10, "polmoni": 9, "rene_sx": 2, "rene_dx": 2, "fegato": 2}



# Esempio di utilizzo della funzione
valore_fo, migliore_t_start = robust_organ_scheduler(
    orari_possibili_inizio_intervento,
    durate_possibili_intervento,
    organi,
    orari_partenza,
    pesi,
)

# Output finale della valutazione robusta
print("Valore funzione obiettivo (minimax):", valore_fo)

if migliore_t_start is not None:
    print("Miglior orario inizio intervento:", migliore_t_start.time())
else:
    print("Miglior orario inizio intervento: nessuna soluzione valida")

Valore funzione obiettivo (minimax): 360.0
Miglior orario inizio intervento: 12:00:00
